# Querying data in NoSQL databases

Objectives:

*   design a data platform that uses MongoDB as a NoSQL database
*   import data into a MongoDB database.
*   query data in a MongoDB database.
*   export data from MongoDB.






Querying data in NoSQL databases:

Task 1 - Import ‘catalog.json’ into mongodb server into a database named ‘catalog’ and a collection named ‘electronics’

Task 2 - List out all the databases

Task 3 - List out all the collections in the database catalog.

Task 4 - Create an index on the field “type”

Task 5 - Write a query to find the count of laptops

Task 6 - Write a query to find the number of smart phones with screen size of 6 inches.

Task 7 - Write a query to find out the average screen size of smart phones.

Task 8 - Export the fields _id, “type”, “model”, from the ‘electronics’ collection into a file named electronics.csv



In [5]:
# prepare .env for credentials
# install python .env
!pip install python-dotenv

# import module 
from dotenv import load_dotenv
import os


load_dotenv()
# print(os.getenv("mongo_user"))



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\fruityty\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


True

Task 1 - Import ‘catalog.json’ into mongodb server into a database named 
‘catalog’ and a collection named ‘electronics’


*   Option 1 import by mongoimport with CLI

mongodb database tool is needed 

install mongodb


```
!apt-get install mongodb
```


CLI command


```
!mongoimport --uri mongodb+srv://<USER>:<PASSWORD>@<HOST>/<DATABASE> --collection <COLLECTION> --type <FILETYPE> --file <FILENAME>
```
USER = MongoDB_User

PASSWORD = MongoDB_Password

HOST = MongoDB_Host

DATABASE = 'catalog'

COLLECTION = 'electronics'

FILETYPE = 'json'

FILENAME = 'catalog.json'





In [4]:
# Option 2 import by pymongo
# install pymongo
!pip install pymongo

# import module
import pymongo
from pymongo.mongo_client import MongoClient

uri = "mongodb+srv://{}:{}@{}".format(os.getenv("mongo_user"), os.getenv("mongo_password"), os.getenv("mongo_host"))

# Create a new client and connect to the server
client = MongoClient(uri)

# Test the connection 
try:
    client.server_info()
    print("Connection successful!")
except Exception as e:
    print("Connection unsuccessful. Error:", e)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\fruityty\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Connection successful!


In [ ]:
# import json file to MongoDB
# import module
import json

# select DB
db = client.catalog

# select collection
collection = db.electronics

# open the JSON file and load the data
with open('catalog.json') as file:
    # insert json data line by line
    count = 0
    for line in file:
        data = json.loads(line)
        # insert the data into a collection
        result = collection.insert_one(data)
        # count records
        if result.inserted_id:
            count += 1

print(f"Inserted {count} records into the collection.")

Inserted 438 records into the collection.


In [6]:
# Task 2 - List out all the databases
all_databases = client.list_database_names()
print(all_databases)

['catalog', 'sample_airbnb', 'sample_analytics', 'sample_geospatial', 'sample_guides', 'sample_mflix', 'sample_restaurants', 'sample_supplies', 'sample_training', 'sample_weatherdata', 'training', 'admin', 'local']


In [7]:
# Task 3 - List out all the collections in the database catalog.
# select DB
db = client.catalog

# list collections
all_collections = db.list_collection_names()

print(f"All collections are {all_collections}")

All collections are ['electronics']


In [8]:
# Task 4 - Create an index on the field “type”
# select DB and collection
db = client.catalog
collection = db.electronics

# create index
index_name = collection.create_index([('type', pymongo.ASCENDING)])

print(f"Index {index_name} created successfully")

# Print the index information
index_list = collection.list_indexes()
for index in index_list:
    print(index)

Index type_1 created successfully
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('type', 1)])), ('name', 'type_1')])


In [9]:
# Task 5 - Write a query to find the count of laptops
# select DB and collection
db = client['catalog']
collection = db['electronics']

# query count of laptops
count_laptops = collection.count_documents({'type':'laptop'})
print(f"Number of laptops in collection is {count_laptops}")

Number of laptops in collection is 389


In [10]:
# Task 6 - Write a query to find the number of smart phones with screen size of 6 inches.
# select DB and collection
db = client['catalog']
collection = db['electronics']

# query count of laptops
count_phone = collection.count_documents({'type':'smart phone', 'screen size':6})
print(f"Number of smart phones with screen size of 6 inches is {count_phone}")

Number of smart phones with screen size of 6 inches is 8


In [11]:
# Task 7 - Write a query to find out the average screen size of smart phones.
# select DB and collection
db = client['catalog']
collection = db['electronics']

# query average screen size of smart phone
pipeline = [
    {'$match':{'type':'smart phone'}},
    {'$group':{'_id':'smart phone', 'avg_screen':{'$avg':'$screen size'}}}
]

avg_screen = collection.aggregate(pipeline).next()['avg_screen']
print(f"The average screen size of smart phones is {avg_screen}")


The average screen size of smart phones is 6.0


Task 8 - Export the fields _id, “type”, “model”, from the ‘electronics’ collection into a file named electronics.csv

*   Option 1 export by mongoexport with CLI


CLI command


```
!mongoexport --uri mongodb+srv://<USER>:<PASSWORD>@<HOST>/<DATABASE> --collection <COLLECTION> --type <FILETYPE> --out <FILENAME> --fields <FIELDS>
```
USER = MongoDB_User

PASSWORD = MongoDB_Password

HOST = MongoDB_Host

DATABASE = 'catalog'

COLLECTION = 'electronics'

FILETYPE = 'csv'

FILENAME = 'electronics.csv'

FIELDS = '_id', 'type', 'model'


In [12]:
# Task 8 - Export the fields _id, “type”, “model”, from the ‘electronics’ collection into a file named electronics.csv
# option 2 export by pymongo
# install pandas
!pip install pandas

# import module
import pandas as pd

# select DB and collection
db = client['catalog']
collection = db['electronics']

# Define the fields to export
fields = {"_id": 1, "type": 1, "model": 1}

# Retrieve the data from MongoDB and create a DataFrame
query = collection.find({}, projection=fields)

# convert find into pandas DataFrame
df = pd.DataFrame(list(query))
df.to_csv('electronics.csv')

# read csv to check
dff = pd.read_csv('electronics.csv')
print(dff)


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\fruityty\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     Unnamed: 0                       _id         type    model
0             0  645e0bb45d6683ac8d61bf50  smart phone  platina
1             1  645e0bb45d6683ac8d61bf51  smart phone     bn20
2             2  645e0bb45d6683ac8d61bf52  smart phone       cz
3             3  645e0bb45d6683ac8d61bf53  smart phone     xm23
4             4  645e0bb55d6683ac8d61bf54  smart phone       c3
..          ...                       ...          ...      ...
433         433  645e0c135d6683ac8d61c101   television      dj1
434         434  645e0c135d6683ac8d61c102   television      dj1
435         435  645e0c145d6683ac8d61c103   television      dj1
436         436  645e0c145d6683ac8d61c104   television      ss2
437         437  645e0c145d6683ac8d61c105   television      dj1

[438 rows x 4 columns]


In [40]:
# close the server connecton
print("Closing the connection.")
client.close()

Closing the connection.
